In [ ]:
!pip uninstall keras -y

In [ ]:
!pip install keras==2.7.0

In [ ]:
pip install --upgrade pip

In [1]:
# display some images for every different expression

import numpy as np
import seaborn as sns
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import os

# size of the image: 48*48 pixels
pic_size = 48

# input path for the images
base_path = "../input/simple-face-expressions/face_expression_simple"



## Image augmentation using keras ImageDataGenerator

# Defining our 4 Convolution and 2 Dense layers model

In [2]:
# modeling
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense,Flatten

from tensorflow.keras.applications import resnet
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping,LearningRateScheduler


from tensorflow.keras import backend as K
K.clear_session()

In [4]:
# building data generator 

from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32
base_path = "../input/simple-face-expressions/face_expression_simple/"



train_datagen = ImageDataGenerator()
#                                   rescale = 1.0/255.0,
#                                   width_shift_range = 0.1,
#                                    height_shift_range = 0.1,
#                                    rotation_range = 20,
#                                    horizontal_flip = True)

validation_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(base_path + "train",
                                                    target_size=(56,56),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(base_path + "validation",
                                                    target_size=(56,56),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

Found 27236 images belonging to 3 classes.
Found 6955 images belonging to 3 classes.


In [7]:
from tensorflow.keras.applications import resnet
#loading resent 
resNet =         resnet.ResNet50(weights = 'imagenet',
                        include_top = False,
                        input_shape = (56,56, 3))

resNet.trainable = False # Freeze layers
resNet_model = Sequential([
        resNet,
        Flatten(),
        Dense(1024, activation = 'relu'),
        Dropout(0.5),
        Dense(3, activation = 'softmax')])
     

optimizer = optimizers.Adam(1e-5)

In [8]:
epoch = 50
learning_rate = 3e-5 
lr_start = 0.00000001
lr_min = 0.000001
lr_max = 3e-5 
lr_rampup_epochs = 1
lr_sustain_epochs = 1
lr_exp_decay = .8

def lrfn(epoch):
    if epoch < lr_rampup_epochs:
        lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
    elif epoch < lr_rampup_epochs + lr_sustain_epochs:
        lr = lr_max
    else:
        lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
    return lr


# checkpoint to save best model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.callbacks import CallbackList 

checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

earlystop = EarlyStopping(patience= 5)
    
lr_callback = LearningRateScheduler(lrfn, verbose = True)

callbacks_list =  [checkpoint,earlystop, lr_callback]

In [10]:
resNet_model.compile(optimizer = optimizer,
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])


# number of epochs to train the NN
epochs = 50


resnet_history = resNet_model.fit_generator(generator=train_generator,
                                steps_per_epoch=train_generator.n//train_generator.batch_size,
                                epochs=epochs,
                                validation_data = validation_generator,
                                validation_steps = validation_generator.n//validation_generator.batch_size,
                                callbacks= callbacks_list)


Epoch 00001: LearningRateScheduler reducing learning rate to 1e-08.
Epoch 1/50
851/851 [==============================] - ETA: 0s - loss: 0.9845 - accuracy: 0.5719
Epoch 00001: val_accuracy improved from 0.57892 to 0.59188, saving model to model_weights.h5
851/851 [==============================] - 33s 39ms/step - loss: 0.9845 - accuracy: 0.5719 - val_loss: 0.9131 - val_accuracy: 0.5919 - lr: 1.0000e-08

Epoch 00002: LearningRateScheduler reducing learning rate to 3e-05.
Epoch 2/50
851/851 [==============================] - ETA: 0s - loss: 0.9348 - accuracy: 0.5824
Epoch 00002: val_accuracy improved from 0.59188 to 0.61550, saving model to model_weights.h5
851/851 [==============================] - 32s 37ms/step - loss: 0.9348 - accuracy: 0.5824 - val_loss: 0.8613 - val_accuracy: 0.6155 - lr: 3.0000e-05

Epoch 00003: LearningRateScheduler reducing learning rate to 3e-05.
Epoch 3/50
850/851 [============================>.] - ETA: 0s - loss: 0.8011 - accuracy: 0.6437
Epoch 00003: val_ac

In [11]:
resNet_model.save("face_resnet_model.h5")

In [ ]:
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam

# number of possible label values
nb_classes = 7

# Initialising the CNN
model = Sequential()

# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(56, 56,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 2nd Convolution layer
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3rd Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 4th Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flattening
model.add(Flatten())

# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(nb_classes, activation='softmax'))

print(model.summary())

opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
%%time

# number of epochs to train the NN
epochs = 50

# checkpoint to save best model
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

history = model.fit_generator(generator=train_generator,
                                steps_per_epoch=train_generator.n//train_generator.batch_size,
                                epochs=epochs,
                                validation_data = validation_generator,
                                validation_steps = validation_generator.n//validation_generator.batch_size,
                                callbacks=callbacks_list
                                )

In [ ]:
# visualise training and testing accuracy and loss

def plot_results(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.figure(figsize = (24, 6))
    plt.subplot(1,2,1)
    plt.plot(epochs, acc, 'b', label = 'Training Accuracy')
    plt.plot(epochs, val_acc, 'r', label = 'Validation Accuracy')
    plt.grid(True)
    plt.legend()
    plt.xlabel('Epoch')
    


    plt.subplot(1,2,2)
    plt.plot(epochs, loss, 'b', label = 'Training Loss')
    plt.plot(epochs, val_loss, 'r', label = 'Validation Loss')
    plt.grid(True)
    plt.legend()
    plt.xlabel('Epoch')
    plt.show()
 
# print best epoch with best accuracy on validation

def get_best_epcoh(history):
    valid_acc = history.history['val_accuracy']
    best_epoch = valid_acc.index(max(valid_acc)) + 1
    best_acc =  max(valid_acc)
    print('Best Validation Accuracy Score {:0.5f}, is for epoch {}'.format( best_acc, best_epoch))
    return best_epoch

In [ ]:
plot_results(history)
best_epoch =get_best_epcoh(history)

## Connecting with openCV

In [ ]:
json_model = model.to_json()

In [ ]:
import json

In [ ]:
print(json_model)

In [ ]:
import json
with open('../input/face-expression-model/model.json','r') as json_file:
    json.read(json_file)

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import model_from_json

In [ ]:
model_json_file = '../input/face-expression-model/model.json'
model_weights_file = '../input/face-expression-model/model_weights.h5'
with open(model_json_file, "r") as json_file:
    loaded_model_json = json_file.read()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(model_weights_file)
    
    

In [ ]:
face_cascade = cv2.CascadeClassifier('../input/face-expression-model/haarcascade_frontalface_default.xml')

In [ ]:
cap = cv2.VideoCapture(0)
import copy

while True:
    
    ret, frame = cap.read()
    img = copy.deepcopy(frame)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
#     for (x,y,w,h) in faces:
#         fc = gray[y:y+h, x:x+w]
        
#         roi = cv2.resize(fc, (48,48))
#         pred = loaded_model.predict(roi[np.newaxis, :, :, np.newaxis])
#         text_idx=np.argmax(pred)
#         text_list = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
#         if text_idx == 0:
#             text= text_list[0]
#         if text_idx == 1:
#             text= text_list[1]
#         elif text_idx == 2:
#             text= text_list[2]
#         elif text_idx == 3:
#             text= text_list[3]
#         elif text_idx == 4:
#             text= text_list[4]
#         elif text_idx == 5:
#             text= text_list[5]
#         elif text_idx == 6:
#             text= text_list[6]
#         cv2.putText(img, text, (x, y-5),
#            cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 0, 255), 2)
#         img = cv2.rectangle(img, (x,y), (x+w, y+h), (0,0,255), 2)
            
    
    cv2.imshow("frame", img)
    key = cv2.waitKey(1) & 0xFF
    if key== ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
